# Trigger files
This notebook is intended to create the trigger files for the eeg data analysis.

In [19]:
import copy
import datetime
import itertools
import os

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from matplotlib.ticker import FormatStrFormatter

import pandas as pd
import pyxdf
import seaborn as sns
from scipy.signal import find_peaks
from tqdm.notebook import tqdm

In [204]:
# path to data stored
r_path = "data"
# path to store trigger data
t_path = r_path + '/triggers'
files = os.listdir(r_path)  # get all files from the folder "data"
files.sort()  # sort them alphabetically
recordings = {}
for i, file in enumerate(files):  # store and display all files
    if file.endswith('.xdf'):
        created = os.path.getmtime(f"data/{file}")  # creation timestamp
        created = datetime.datetime.fromtimestamp(created)  # translate as datetime
        created = created.strftime("%d.%m.%Y %H:%M")  # arrange it
        recordings[i] = {"file": file, "created": created}

files = [f.split(".")[0] for f in files]
print("Included:")
display(recordings)

Included:


{1: {'file': '39_room1_251022.xdf', 'created': '25.10.2022 16:40'},
 2: {'file': '40_room1_261022.xdf', 'created': '26.10.2022 16:54'}}

## 1. Load data

In [154]:
# check streams for recording 0
streams, _ = pyxdf.load_xdf(f"data/{recordings[1]['file']}")

In [155]:
# stream channel names in recording 0
s_channels = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}
s_channels

{'HitPositionOnObjects': 0,
 'ImagesOrder': 1,
 'HitObjectPositions': 2,
 'Visual': 3,
 'ImageInfo': 4,
 'ValidationError': 5,
 'EyeTrackingLocal': 6,
 'HeadTracking': 7,
 'openvibeSignal': 8,
 'EyeTrackingWorld': 9,
 'HitObjectNames': 10}

In [208]:
streams[8]

{'info': defaultdict(list,
             {'name': ['openvibeSignal'],
              'type': ['signal'],
              'channel_count': ['72'],
              'channel_format': ['float32'],
              'source_id': ['(0x0e2acdb6, 0xbdd6629e)'],
              'nominal_srate': ['1024.000000000000'],
              'version': ['1.100000000000000'],
              'created_at': ['1020870.363311500'],
              'uid': ['27a4685b-41c6-4292-b473-676c682130ec'],
              'session_id': ['default'],
              'hostname': ['wd-left'],
              'v4address': [None],
              'v4data_port': ['16572'],
              'v4service_port': ['16572'],
              'v6address': [None],
              'v6data_port': ['16572'],
              'v6service_port': ['16572'],
              'desc': [defaultdict(list,
                           {'channels': [defaultdict(list,
                                         {'channel': [defaultdict(list,
                                                    

In [23]:
#
def select_streams(streams):
    # stream names
    names_ch = "ImageInfo"
    # e_ch_name = "openvibeSignal"

    # get all current streams with their positions on the recording
    # example: {'Diode': 0, 'Audio': 1, 'openvibeSignal': 2}
    s_channels = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}

    # store and return their positions
    u = s_channels[names_ch]
    # e = s_channels[e_ch_name]  # eeg stream channel (diode and microphone)
    return u


## 2. Create dataframe from streams

In [165]:
def get_streams_data(streams, streams_keep=['ImageInfo','Visual']):
    """
    :param streams: streams after loading from .xdf file
    :param streams_keep: str. of the stream names to keep
    :return: df containing the time_stamps and stream_data as columns for each stream to keep
    """
    data = pd.DataFrame()
    for i, ch_name in enumerate(streams_keep):
        # get all current streams with their positions on the recording
        # example: {'ImagesOrder': 0, 'ValidationError': 1, 'HeadTracking': 2}
        s_channels = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}
        u = s_channels[ch_name]
        # save the subject UID and append to df
        uid = streams[u]['info']['uid']
        data['UID'] = np.resize(uid,len(data))

        # check the type and length of data arrays and get only 1 value of the array
        stream_data = streams[u]['time_series']
        # double check keys on each stream to make sure they are all appended to df
        print(f"Stream {ch_name} keys: {streams[u]['info']['desc'][0].keys()}")
        # check stram_data is of kind np.array()
        if isinstance(stream_data, (list,pd.core.series.Series,np.ndarray)):
            # access all stream names in dictionary's 'info' description
            for i, key in enumerate(streams[u]['info']['desc'][0].keys()):
                # save each dict key as column to df
                stream_data = pd.DataFrame(streams[u]['time_series'])[i]
                data[f"{key}_{ch_name}"] = stream_data

        # get timestamps and attach them as column to df
        time_stamps =  streams[u]['time_stamps']
        data = pd.concat([data, pd.DataFrame(time_stamps, columns=[f"time_stamps_{ch_name}"])], axis=1)

    return data

In [166]:
df = get_streams_data(streams, streams_keep=['ImageInfo','Visual'])
df

Stream ImageInfo keys: dict_keys(['blockNumber', 'imageName'])
Stream Visual keys: dict_keys(['cFrame', 'displayStatus', 'worldTime'])


,UID,blockNumber_ImageInfo,imageName_ImageInfo,time_stamps_ImageInfo,cFrame_Visual,displayStatus_Visual,worldTime_Visual,time_stamps_Visual
0,dfb99d79-4595-4a0d-b346-23282e000f10,0,startMessage,1.021258e+06,1831.0,-1.0,57.157524,1.021258e+06
1,dfb99d79-4595-4a0d-b346-23282e000f10,0,startMessage,1.021258e+06,1832.0,-1.0,57.162384,1.021258e+06
2,dfb99d79-4595-4a0d-b346-23282e000f10,0,startMessage,1.021258e+06,1832.0,-1.0,57.163330,1.021258e+06
3,dfb99d79-4595-4a0d-b346-23282e000f10,0,startMessage,1.021258e+06,1832.0,-1.0,57.163502,1.021258e+06
4,dfb99d79-4595-4a0d-b346-23282e000f10,0,startMessage,1.021258e+06,1832.0,-1.0,57.163681,1.021258e+06
...,...,...,...,...,...,...,...,...
345912,dfb99d79-4595-4a0d-b346-23282e000f10,3,endMessage,1.025199e+06,347754.0,99.0,3998.594971,1.025199e+06
345913,dfb99d79-4595-4a0d-b346-23282e000f10,3,endMessage,1.025199e+06,347755.0,99.0,3998.606201,1.025199e+06
345914,dfb99d79-4595-4a0d-b346-23282e000f10,3,endMessage,1.025199e+06,347756.0,99.0,3998.617188,1.025199e+06
345915,dfb99d79-4595-4a0d-b346-23282e000f10,3,endMessage,1.025199e+06,347757.0,99.0,3998.628174,1.025199e+06


## 3. Create triggers
- For each initial time an image was shown, we want to keep the type of object it was (i.e., face, object, body) as a separate column.
- Additional triggers contain the rotation and distance the specific object was with respect to the player at the time the free-viewing walk took place.
- __Note:__ We want the triggers only once to denote the initial time the image was shown.

In [199]:
def create_triggers(df):
    # save the names of the image category
    df['Name'] =  df.apply(lambda x: x["imageName_ImageInfo"].split(".")[5] if len(x["imageName_ImageInfo"].split(".")) > 7 else '', axis=1)
    # creating the triggers for the first time an image is shown
    # check when there is a change from image, canvas, fixationCross
    df['shift'] = df['imageName_ImageInfo'].shift(1) != df['imageName_ImageInfo']
    df['time_onset'] = df.apply(lambda x: x['time_stamps_Visual'] if len(x["imageName_ImageInfo"].split(".")) > 7 and x['shift'] else '', axis=1)
    df['triggers'] = df.apply(lambda x: 'face' if x['shift'] and 'face' in x['imageName_ImageInfo'].lower()
                                               else ('body' if x['shift'] and 'npc' in x['imageName_ImageInfo'].lower()
                                               else ('object' if x['shift'] and 'rotation' in x['imageName_ImageInfo'].lower() and 'face|npc' not in x['imageName_ImageInfo'].lower()
                                               else '')), axis=1)
    # define the triggers for rotation and distance
    df['rotation'] = df.apply(lambda x: 'r_' + x["imageName_ImageInfo"].split(".")[7] if len(x["imageName_ImageInfo"].split(".")) > 7 and x['shift'] else '', axis=1)
    df['distance'] = df.apply(lambda x: 'd_' + x["imageName_ImageInfo"].split(".")[9] if len(x["imageName_ImageInfo"].split(".")) > 7 and x['shift'] else '', axis=1)
    df['block'] = df.apply(lambda x: 'b_' + str(x["blockNumber_ImageInfo"]) if len(x["imageName_ImageInfo"].split(".")) > 7 and x['shift'] else '', axis=1)
    df_sel = df[['time_onset','triggers','rotation','distance','block']]
    df_triggers = df_sel[df_sel['time_onset'] != '']
    uid = df['UID'][0]
    return df_triggers, uid

In [200]:
df_triggers, uid = create_triggers(df)
df_triggers

,time_onset,triggers,rotation,distance,block
1726,1021276.707802,body,r_117,d_5,b_0
1873,1021278.34043,object,r_49,d_7,b_0
2019,1021279.961899,face,r_29,d_7,b_0
2189,1021281.861035,face,r_72,d_3,b_0
2345,1021283.593613,face,r_14,d_3,b_0
...,...,...,...,...,...
344807,1025186.804562,body,r_109,d_9,b_3
344967,1025188.581551,body,r_54,d_5,b_3
345148,1025190.591745,body,r_5,d_1,b_3
345294,1025192.213329,body,r_54,d_5,b_3


In [226]:
for file in os.listdir(r_path):
    if file.lower().endswith('.xdf'):
        print(os.path.join(r_path, file))
        # check streams for recording 0
        streams, _ = pyxdf.load_xdf(os.path.join(r_path, file))
        # store stream data into df
        df = get_streams_data(streams, streams_keep=['ImageInfo','Visual'])
        df_triggers, uid = create_triggers(df)
        if not os.path.isdir(t_path):
            os.mkdir(t_path)
        df_triggers.to_csv(os.path.join(t_path, 'trigger_file_' + uid +'.csv'), index=False)


data/39_room1_251022.xdf
Stream ImageInfo keys: dict_keys(['blockNumber', 'imageName'])
Stream Visual keys: dict_keys(['cFrame', 'displayStatus', 'worldTime'])
data/40_room1_261022.xdf
Stream ImageInfo keys: dict_keys(['blockNumber', 'imageName'])
Stream Visual keys: dict_keys(['cFrame', 'displayStatus', 'worldTime'])


'/Users/johnmadrid/GitHub/eeg_analysis/data'

In [220]:
os.getcwd()[-5:]

'/data'

In [224]:
os.chdir('/Users/johnmadrid/GitHub/eeg_analysis/')